In [1]:
from aocd import submit
import numpy as np
import re
from itertools import product

In [2]:
YYYY=2021
DD=22

In [3]:
def translate(s):
    parse=re.compile("[-+]?[0-9]+")
    s1 = 1 if  s[0:2] == "on" else 0
    s2 = [int(n) for n in parse.findall(s)]
    s2 = [(s2[0], s2[1]), (s2[2], s2[3]), (s2[4], s2[5])]

    return [s1, tuple(s2)]
    
def load(filename):
    with open(filename,'r') as f:
         lines = [translate(s)  for s in f]
    return lines
test1 = load("test1.dat")
test2 = load("test2.dat")
test2

[[1, ((-20, 26), (-36, 17), (-47, 7))],
 [1, ((-20, 33), (-21, 23), (-26, 28))],
 [1, ((-22, 28), (-29, 23), (-38, 16))],
 [1, ((-46, 7), (-6, 46), (-50, -1))],
 [1, ((-49, 1), (-3, 46), (-24, 28))],
 [1, ((2, 47), (-22, 22), (-23, 27))],
 [1, ((-27, 23), (-28, 26), (-21, 29))],
 [1, ((-39, 5), (-6, 47), (-3, 44))],
 [1, ((-30, 21), (-8, 43), (-13, 34))],
 [1, ((-22, 26), (-27, 20), (-29, 19))],
 [0, ((-48, -32), (26, 41), (-47, -37))],
 [1, ((-12, 35), (6, 50), (-50, -2))],
 [0, ((-48, -32), (-32, -16), (-15, -5))],
 [1, ((-18, 26), (-33, 15), (-7, 46))],
 [0, ((-40, -22), (-38, -28), (23, 41))],
 [1, ((-16, 35), (-41, 10), (-47, 6))],
 [0, ((-32, -23), (11, 30), (-14, 3))],
 [1, ((-49, -5), (-3, 45), (-29, 18))],
 [0, ((18, 30), (-20, -8), (-3, 13))],
 [1, ((-41, 9), (-7, 43), (-33, 15))],
 [1, ((-54112, -39298), (-85059, -49293), (-27449, 7877))],
 [1, ((967, 23432), (45373, 81175), (27513, 53682))]]

In [4]:
data_in = load("input.dat")

# Teil a

In [5]:
"""
0..100
-50 = 0
50 = 100
"""
a= 'on x=10..12,y=10..12,z=10..12'
parse=re.compile("\d+")
parse.findall(a)

['10', '12', '10', '12', '10', '12']

In [6]:
def transform(cubes):
    c=list(cubes)
    for i,v in enumerate(cubes):
        c[i] = 50+(min(50, v) if i % 2 else max(-50,v))
    return c

In [7]:
transform(test1[0][1])

[60, 62, 60, 62, 60, 62]

In [8]:
def solution_a(data):
    c = np.zeros((101,101,101), int)
    for s, d in data:
        x1,x2, y1,y2, z1, z2 = transform(d)
        c[x1:x2+1,y1:y2+1,z1:z2+1] = s
        #print(c.sum())
    return c.sum()

In [9]:
solution_a(test2)

590784

In [10]:
submit(solution_a(data_in),part="a",day=DD, year=YYYY)

Part a already solved with same answer: 553201


# Teil b

In [19]:
a=list((3,2,1))
a.sort()
a

[1, 2, 3]

In [193]:
def overlap(a,b):
    # a1 < a2
    a1,a2=a
    b1,b2=b
    return not (a1 >= b2 or b1 >= a2)

def count_overlap(c1, c2):
    x1, y1, z1 = c1
    x2, y2, z2 = c2
    if overlap(x1,x2) and overlap(y1,y2) and overlap(z1,z2):
        x = (max(x1[0], x2[0]), min(x1[1],x2[1]))
        y = (max(y1[0], y2[0]), min(y1[1],y2[1]))
        z = (max(z1[0], z2[0]), min(z1[1],z2[1]))
        return (1+x[1]-x[0])*(1+y[1]-y[0])*(1+z[1]-z[0])
    return 0

def _isin(a, b):
    # is a in b?
    a1,a2=a
    b1,b2=b
    return (a1 >= b1 and a2 <= b2)    

def isin(a, b):
    # is a in b?
    x1,y1,z1 = a
    x2,y2,z2 = b
    return _isin(x1, x2) and _isin(y1,y2) and _isin(z1,z2)


def mintuple(l):
    values = {b[0] for b in l}
    result = []
    for v in values:
        result.append(min([b for b in l if b[0]==v], key=lambda x:x[1]))
    result.sort()
    return result

def legal(a,b):
    x = mintuple([ (n,m) for n, m in product(a+b, a+b) if n<m])
    for i, xx in enumerate(x[:-1]):
        if x[i][1] == x[i+1][0]:
            x[i] = (x[i][0], x[i][1]-1)
    return x

def split(c1 , c2):
    x1, y1, z1 = c1
    x2, y2, z2 = c2
    if overlap(x1,x2) and overlap(y1,y2) and overlap(z1,z2):
        x = legal(x1,x2)
        y = legal(y1,y2)
        z = legal(z1,z2)
        return [(c[0][0], c[0][1], c[1]) for c in product(product(x,y),z)]
    else:
        return [c1,c2]

def merge(l, cube):
    result=set()
    for c in l:
        result |= set(split(c, cube))
    return result
    
def get_countable(cube, subcube):
    x,y,z = cube
    xx,yy,zz = list(subcube) 
    if xx[1] < x[1]:
        xx = (xx[0], xx[1] - 1)
    if yy[1] < y[1]:
        yy = (yy[0], yy[1] - 1)
    if zz[1] < z[1]:
        zz = (zz[0], zz[1] - 1)
    return (1+xx[1]-xx[0])*(1+yy[1]-yy[0])*(1+zz[1]-zz[0])

def count(subcube):
    xx,yy,zz = subcube
    return (1+xx[1]-xx[0])*(1+yy[1]-yy[0])*(1+zz[1]-zz[0])

    

In [147]:
%%script echo "skip"
def calc_subcubes(data):
    cubes = {data[0][1]}
    for s,c in data[1:]:
        cubes = merge(cubes, c)
    return {c: 0 for c in cubes}

splitted=calc_subcubes(test2)
len(splitted)


skip


In [198]:
def solution_b(data):
    on = 0
    splitted=calc_subcubes(data)
    for s, cube in data:
        # have the cube to perform
        print(cube)
        for k,v in splitted.items():
            if isin(k, cube):
                splitted[k] = s
    for k,v in splitted.items():
        if v == 1:
            print(f"{k, count(k)}")
            on += count(k)
    return on
    

In [203]:
solution_b([test1[0], test1[3]])

((10, 12), (10, 12), (10, 12))
((10, 10), (10, 10), (10, 10))
(((10, 10), (10, 10), (10, 10)), 1)
(((10, 12), (10, 12), (10, 12)), 27)


28

In [204]:
legal((10, 12), (10, 10))

[(10, 12)]

In [114]:
113-39

74

In [5]:


def extract(rule, wide):
    (x1, x2), (y1, y2), (z1, z2)= rule
    cubes = set()
    for x in range(floorx(x1,wide), ceilx(x2,wide), wide):
        for y in range(floorx(y1,wide), ceilx(y2,wide), wide):
            for z in range(floorx(z1,wide), ceilx(z2,wide), wide):
                cubes.add((x,y,z))
    return cubes

def ceilx(d,wide):
    return int(wide*np.ceil(d/wide))

def floorx(d, wide):
    return int(wide*np.floor(d/wide))

def minmax(d,wide):
    nx,mx = 1000,-1000
    ny,my = 1000,-1000
    nz,mz = 1000,-1000
    for s, dd in d:
        (x1,x2), (y1,y2),(z1,z2) = dd
        nx, mx = min(nx, x1), max(mx,x2)
        ny, my = min(ny, x1), max(my,x2)
        nz, mz = min(nz, x1), max(mz,x2)
    return floorx(nx,wide),ceilx(mx,wide),floorx(ny,wide),ceilx(my,wide),floorx(nz,wide),ceilx(mz, wide)
    #return nx, mx, ny, my, nz,mz

def dim(nx, mx, ny, my, nz,mz):
    return abs(nx)+abs(mx)+1, abs(ny)+abs(my)+1, abs(nz)+abs(mz)+1

def transform(cubes, x1, x2, y1, y2, z1, z2, r=100): # x1... means the offsets
    c=[0]*6
    c[0] = -x1+cubes[0][0]
    c[1] = -x1+cubes[0][1]
    c[2] = -y1+cubes[1][0]
    c[3] = -y1+cubes[1][1]
    c[4] = -z1+cubes[2][0]
    c[5] = -z1+cubes[2][1]
    if any([x > r for x in c[::2]]) or any([x < 0 for x in c[1::2]]):
        return [-1,-1,-1,-1,-1,-1]
    return [min(r,max(0,x)) for x in c]


In [6]:
def solution_b(data):
    s=0
    wide = 500
    nx, mx, ny, my, nz,mz = minmax(data, wide)
    #nx, mx, ny, my, nz,mz = (-50,50,-50,50,-50,50)
    print(nx, mx, ny, my, nz,mz)
    #nx,ny,nz=0,0,0
    mycubes = set()
    for switch, d in data:
        mycubes |= extract(d,wide)
    print(f"identified {len(mycubes)} cubes",flush=True)
    for i, (x,y,z) in enumerate(mycubes):
        if i%100000 == 0:
            print(f"cube {i}: {(x,y,z)}", flush=True)
        c = np.zeros((wide+1,wide+1,wide+1), np.bool)
        # c[:,:,:] = 0
        for switch, d in data: 
            x1,x2, y1,y2, z1, z2 = transform(d, x,x+wide,y,y+wide,z,z+wide)
            if x1 == -1:
                continue
            #print(f"{d} -> {x1,x2+1,y1,y2+1,z1,z2+1}")
            c[x1:x2+1,y1:y2+1,z1:z2+1] = switch
        s += c.sum()
        if i%100000 == 0:
            print(s, flush=True)
    return s

In [7]:
solution_b(data_in)

-98000 93000 -98000 93000 -98000 93000
identified 14937980 cubes
cube 0: (58000, 14500, -62000)
1030301


/home/grisuji/anaconda3/envs/aoc/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app


KeyboardInterrupt: 

In [9]:
200000**2

40000000000

In [8]:
a=np.zeros((200000, 200000))

MemoryError: Unable to allocate 298. GiB for an array with shape (200000, 200000) and data type float64

In [ ]:
submit(solution_b(data_in),part="b",day=DD, year=YYYY)